In [1]:
############################################################################
##
## Copyright (C) 2021 NVIDIA Corporation.  All rights reserved.
##
## NVIDIA Sample Code
##
## Please refer to the NVIDIA end user license agreement (EULA) associated
## with this source code for terms and conditions that govern your use of
## this software. Any use, reproduction, disclosure, or distribution of
## this software and related documentation outside the terms of the EULA
## is strictly prohibited.
##
############################################################################

### Set the dir location to operate in.

In [6]:
# coalesce a large number of price vector files in NASDAQ and NYSE dir into one data frame

import csv, os, sys,time, datetime
import pandas as pd, numpy as np, cudf
from matplotlib import pyplot as plt
#replace dir with the location on your machine where NYSE and NASDAQ subdirs exist
dir = '/raid/mbennett/GPU/widedf/'
os.chdir(dir)
dir

'/raid/mbennett/GPU/widedf/'

### Set gpuNum at 0 or 1 for GPU-acceleration. Look inside lab.csv to obtain all ticker symbols

In [7]:
gpuNum = 1
if gpuNum == 0:
    from pandas import DataFrame,Series,read_csv,concat,unique,merge,notnull
    from numpy import isnan
else:
    from cudf import DataFrame,Series,read_csv,concat,from_pandas,merge
    from cupy import isnan
    pass

if gpuNum == 0:
    colList = read_csv('lab.csv',header=None)[0].values.tolist()
else:
    colList = read_csv('lab.csv',header=None)['0'].to_arrow().to_pylist()
colList.insert(0,'Date')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print(len(colList),'total securities')

6072 total securities


### 2-pass algorithm: only some of the cols will exist in a given file due to the NYSE/NASDAQ split.
### 1. Form all desired unique times from all NYSE/NASDAQ files
### 2. Go back to all NYSE/NASDAQ file and extract columns

In [10]:
matchStrNASDAQ = 'NASDAQ/data/NASDAQ_2018'; lenNAS = len(matchStrNASDAQ)
matchStrNYSE   = 'NYSE/data/NYSE_2018';     lenNYS = len(matchStrNYSE)

def getUniqueTimes(fileList,cols):
    #iterate through all files to determine all expected date and time keys:
    dateDF = pd.DataFrame(columns=['Date'])
    ###if gpuNum > 0:  dateDF = from_pandas(dateDF)
    for file in fileList:
        if (file[0:lenNAS] == matchStrNASDAQ) or (file[0:lenNYS] == matchStrNYSE):
            print('Reading...')
            DF = pd.read_csv(file)
            #whittle down to what we need: Date
            print('Striping Date...')
            DF = DF[['Date']]
            print('Appending Dates...')
            dateDF = pd.concat([dateDF,DF])
            print(file,dateDF.shape)
    print('Sorting...')
    dateVec = dateDF['Date'].unique()
    dateVec.sort()
    df = pd.DataFrame(pd.Series(dateVec),columns=['Date'])
    if gpuNum > 0: df = cudf.DataFrame.from_pandas(df)
    return df

### Goal is to form a date data frame, dateDF, having a comprehensive column of the dates and times that we need to attach the prices to.
### This dateDF is based upon all the files appearing in ./NASDAQ/data and ./NYSE/data.

In [12]:
fileList1 = ['NASDAQ/data/{0}'.format(i) for i in os.listdir(dir + 'NASDAQ/data')]
fileList1.sort()
fileList1.pop(0)
fileList2 = ['NYSE/data/{0}'.format(i)   for i in os.listdir(dir + 'NYSE/data')]
fileList2.sort()
fileList2.pop(0)
#unit test getUniqueTimes():
#if False: #test case only
#    fileList1 = ['NASDAQ/data/NASDAQ_20180102_1.csv']
#    fileList2 = ['NYSE/data/NYSE_20180102_1.csv']

fileList  = fileList1 + fileList2
print('files:', fileList)
print('file count:', len(fileList))
cols = colList[1:len(colList)]

dateDF = getUniqueTimes(fileList,cols)

files: ['NASDAQ/data/NASDAQ_20181203_1.csv', 'NASDAQ/data/NASDAQ_20181204_1.csv', 'NASDAQ/data/NASDAQ_20181206_1.csv', 'NASDAQ/data/NASDAQ_20181207_1.csv', 'NASDAQ/data/NASDAQ_20181210_1.csv', 'NASDAQ/data/NASDAQ_20181211_1.csv', 'NASDAQ/data/NASDAQ_20181212_1.csv', 'NASDAQ/data/NASDAQ_20181213_1.csv', 'NASDAQ/data/NASDAQ_20181214_1.csv', 'NASDAQ/data/NASDAQ_20181217_1.csv', 'NASDAQ/data/NASDAQ_20181218_1.csv', 'NASDAQ/data/NASDAQ_20181219_1.csv', 'NASDAQ/data/NASDAQ_20181220_1.csv', 'NASDAQ/data/NASDAQ_20181221_1.csv', 'NASDAQ/data/NASDAQ_20181224_1.csv', 'NASDAQ/data/NASDAQ_20181226_1.csv', 'NASDAQ/data/NASDAQ_20181227_1.csv', 'NASDAQ/data/NASDAQ_20181228_1.csv', 'NASDAQ/data/NASDAQ_20181231_1.csv', 'NYSE/data/NYSE_20181030_1.csv', 'NYSE/data/NYSE_20181203_1.csv', 'NYSE/data/NYSE_20181204_1.csv', 'NYSE/data/NYSE_20181206_1.csv', 'NYSE/data/NYSE_20181207_1.csv', 'NYSE/data/NYSE_20181210_1.csv', 'NYSE/data/NYSE_20181211_1.csv', 'NYSE/data/NYSE_20181212_1.csv', 'NYSE/data/NYSE_20181213_

,Date
0,03-Dec-2018 09:00
1,03-Dec-2018 09:01
2,03-Dec-2018 09:02
3,03-Dec-2018 09:03
4,03-Dec-2018 09:04
...,...
9593,31-Dec-2018 16:55
9594,31-Dec-2018 16:56
9595,31-Dec-2018 16:57
9596,31-Dec-2018 16:58


In [13]:
dateDF

,Date
0,03-Dec-2018 09:00
1,03-Dec-2018 09:01
2,03-Dec-2018 09:02
3,03-Dec-2018 09:03
4,03-Dec-2018 09:04
...,...
9593,31-Dec-2018 16:55
9594,31-Dec-2018 16:56
9595,31-Dec-2018 16:57
9596,31-Dec-2018 16:58


### Fill in any NaN prices with the prior price down the column price vectors.
### See unit test for an example.

In [15]:
def fillInPrices(vec):
    #fill in by flattening all NaN prices
    initPrice = vec[0] #find init price
    for i in range(len(vec)):
        price = vec[i]
        if price is not None and not isnan(price) and notnull(price):
            initPrice = price
            break #early break out
    isInitPriceFilled = False  
    savePrice = initPrice
    for i in range(len(vec)):
        #two modes: init section then afterward
        if not isInitPriceFilled:
            if vec[i] is None or isnan(vec[i]) or not notnull(vec[i]):
                vec[i] = initPrice
                isInitPriceFilled = True
        else: #either set NaN price or save non-NaN price
            if vec[i] is None or isnan(vec[i]) or not notnull(vec[i]):
                vec[i] = savePrice
            else:
                savePrice = vec[i]
    return vec
#unit test:
vec = [np.NaN,13.2,np.NaN,None,12.9]
fillInPrices(vec) == [13.2, 13.2, 13.2, 13.2, 12.9]

True

### For the ticker symbol set of interest, extract the prices for those columns in main data frame, DF.
### use concat([mainDF, rightDF],axis=1) repeatedly to right append columns to mainDF.
### Finally write mainDF to prices.csv in the directory.

In [16]:
start = time.time()

def extractCols(fileList,cols,dateDF):
    #iterate through all files and attach them to proper time rows in cols
    mainDF = dateDF #dateDF is a dateDF.shape[0] by 1 init DF
    for col in cols:
        print(col,"=============")
        #whittle down to what we need: Symbol, Date, Close
        DF = DataFrame(columns=['Symbol', 'Date','Close'])
        for file in fileList:
            if (file[0:lenNAS] == matchStrNASDAQ) or (file[0:lenNYS] == matchStrNYSE):
                fileDF = read_csv(file)
                fileDF = fileDF[['Symbol', 'Date','Close']]
                fileDF = fileDF[(fileDF.Symbol == col)]
                print(file,mainDF.shape)
                if fileDF.shape[0] != 0:
                    #DF = DF.append(fileDF)
                    DF = concat([DF,fileDF])
                else:
                    print("no row appended")
        tickerDF = DF[DF['Symbol'].str.match(col)]
        print('tickerDF',tickerDF)
        if tickerDF.shape[0] != 0 and not col in mainDF.columns:
            #make right frontier of mainDF look like tickerDF to join : CLose was Time below
            data = {'Symbol':[col] * dateDF.shape[0], 'Close':[np.NaN] * dateDF.shape[0]} 
            rightDF = DataFrame(data) 
            holdColNames = mainDF.columns.values
            mainDF = concat([mainDF, rightDF],axis=1)
            print('mainDF',mainDF)
            '''
            ####mainDF = concat([mainDF, rightDF], axis=1, ignore_index=True)
            #What value is sitting in the first row last col?
            ####if mainDF.iloc[0][mainDF.shape[1]-1]==col: #pandas issue work around
            if notnull(mainDF['Symbol'].iloc[0]) and mainDF['Symbol'].iloc[0]==col:
                print(holdColNames,type(holdColNames))
                #mainDF.columns = holdColNames+['Close','Symbol']  
                #columnOrder = holdColNames.tolist()+['Close','Symbol']       
            else: #Either way label it properly now:
                print(holdColNames,type(holdColNames))
                #mainDF.columns = holdColNames+['Symbol','Close']
                #columnOrder = holdColNames.tolist()+['Symbol','Close']
            ####for i in range(0,len(columnOrder)):
            ####    mainDF = mainDF.rename(columns={i:columnOrder[i]})    
            '''
            mainDF = merge(mainDF, tickerDF, on=['Symbol','Date'], how='left')
            mainDF = mainDF.sort_values(by='Date')
            close_y_series = mainDF['Close_y']
            if gpuNum > 0: close_y_series = close_y_series.to_pandas()
            mainDF['Close_y'] = fillInPrices(close_y_series.tolist())
            mainDF = mainDF.drop('Close_x', axis=1)
            mainDF.rename(columns = {'Close_y':col}, inplace = True)
            mainDF = mainDF.drop('Symbol', axis=1)
    return mainDF

def plotIncFig(DF,cols):
   fig = plt.figure(figsize=(12, 3))
   fig, axes = plt.subplots(nrows=1, ncols=4)#len(cols))
   axList = axes.flatten()
   axIdx = 0; colors = ['red','tan','lime','blue']*10
   for col in cols[0:4]:
       axList[axIdx].set_title(col)
       axList[axIdx].plot(DF[col].head(15)/10.0,color=colors[axIdx])
       axIdx += 1
       fig.tight_layout()
       plt.show()

####if gpuNum > 0: dateDF = dateDF.to_pandas()
####cols = cols[0:10]
cols = ['Date','AAPL','CSCO','INTC','MSFT','JNJ','JPM','KO','MCD','MMM',
    'NKE','PFE','PG','TRV','UNH','UTX','V','VZ','WMT','XOM',
    'AXP','BA','CAT','CVX','DWDP','DIS','GE','GS','HD','IBM','MRK']
start = time.time()
mainDF = extractCols(fileList,cols,dateDF)
print(mainDF.shape[0],'total simulated minutes time to process is',time.time()-start)
##############################################plotIncFig(mainDF,cols)
print(mainDF)
mainDF.to_csv('prices.csv', index=False, header=True)

print(time.time()-start,'secs.')

Date =============
NASDAQ/data/NASDAQ_20181203_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181204_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181206_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181207_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181210_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181211_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181212_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181213_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181214_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181217_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181218_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181219_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181220_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181221_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181224_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20181226_1.csv (9598, 1)
no row appended
NASDAQ/data/NASDAQ_20

INTC =============
NASDAQ/data/NASDAQ_20181203_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181204_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181206_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181207_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181210_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181211_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181212_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181213_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181214_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181217_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181218_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181219_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181220_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181221_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181224_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181226_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181227_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181228_1.csv (9598, 3)
NASDAQ/data/NASDAQ_20181231_1.csv (9598, 3)
NYSE/data/NYSE_20181030_1.csv (9598, 3)
no row appended
NYSE/data/NYSE_20181203_1.csv (9598, 3)
no row appended
NYSE/data/NYSE_20181204_1.csv (95

JPM =============
NASDAQ/data/NASDAQ_20181203_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181204_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181206_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181207_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181210_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181211_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181212_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181213_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181214_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181217_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181218_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181219_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181220_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181221_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181224_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_20181226_1.csv (9598, 6)
no row appended
NASDAQ/data/NASDAQ_201

NYSE/data/NYSE_20181206_1.csv (9598, 8)
NYSE/data/NYSE_20181207_1.csv (9598, 8)
NYSE/data/NYSE_20181210_1.csv (9598, 8)
NYSE/data/NYSE_20181211_1.csv (9598, 8)
NYSE/data/NYSE_20181212_1.csv (9598, 8)
NYSE/data/NYSE_20181213_1.csv (9598, 8)
NYSE/data/NYSE_20181214_1.csv (9598, 8)
NYSE/data/NYSE_20181217_1.csv (9598, 8)
NYSE/data/NYSE_20181218_1.csv (9598, 8)
NYSE/data/NYSE_20181219_1.csv (9598, 8)
NYSE/data/NYSE_20181220_1.csv (9598, 8)
NYSE/data/NYSE_20181221_1.csv (9598, 8)
NYSE/data/NYSE_20181224_1.csv (9598, 8)
NYSE/data/NYSE_20181226_1.csv (9598, 8)
NYSE/data/NYSE_20181227_1.csv (9598, 8)
NYSE/data/NYSE_20181228_1.csv (9598, 8)
NYSE/data/NYSE_20181231_1.csv (9598, 8)
tickerDF        Symbol               Date   Close
362461    MCD  30-Oct-2018 09:01  173.36
362462    MCD  30-Oct-2018 09:11  173.95
362463    MCD  30-Oct-2018 09:13  173.95
362464    MCD  30-Oct-2018 09:14  173.78
362465    MCD  30-Oct-2018 09:15  173.78
...       ...                ...     ...
376038    MCD  31-Dec-20

PFE =============
NASDAQ/data/NASDAQ_20181203_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181204_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181206_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181207_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181210_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181211_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181212_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181213_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181214_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181217_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181218_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181219_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181220_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181221_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181224_1.csv (9598, 11)
no row appended
NASDAQ/data/NASDAQ_20181226_1.csv (9598, 11)
no row appended
NASDAQ

...

CVX =============
NASDAQ/data/NASDAQ_20181203_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181204_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181206_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181207_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181210_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181211_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181212_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181213_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181214_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181217_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181218_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181219_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181220_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181221_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181224_1.csv (9598, 23)
no row appended
NASDAQ/data/NASDAQ_20181226_1.csv (9598, 23)
no row appended
NASDAQ

DIS =============
NASDAQ/data/NASDAQ_20181203_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181204_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181206_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181207_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181210_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181211_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181212_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181213_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181214_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181217_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181218_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181219_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181220_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181221_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181224_1.csv (9598, 25)
no row appended
NASDAQ/data/NASDAQ_20181226_1.csv (9598, 25)
no row appended
NASDAQ

GS =============
NASDAQ/data/NASDAQ_20181203_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181204_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181206_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181207_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181210_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181211_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181212_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181213_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181214_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181217_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181218_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181219_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181220_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181221_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181224_1.csv (9598, 27)
no row appended
NASDAQ/data/NASDAQ_20181226_1.csv (9598, 27)
no row appended
NASDAQ/

IBM =============
NASDAQ/data/NASDAQ_20181203_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181204_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181206_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181207_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181210_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181211_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181212_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181213_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181214_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181217_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181218_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181219_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181220_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181221_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181224_1.csv (9598, 29)
no row appended
NASDAQ/data/NASDAQ_20181226_1.csv (9598, 29)
no row appended
NASDAQ

9598 total simulated minutes time to process is 62.21324276924133
                   Date    AAPL   CSCO   INTC    MSFT     JNJ     JPM     KO     MCD     MMM    NKE    PFE     PG     TRV     UNH     UTX       V     VZ    WMT    XOM     AXP     BA     CAT     CVX   DWDP     DIS    GE      GS      HD     IBM    MRK
8444  03-Dec-2018 09:00  183.97  48.45  50.07  112.80  147.75  112.94  49.93  190.40  211.36  76.85  46.51  94.73  130.48  285.50  123.50  144.10  59.83  99.00  80.65  113.59  361.0  141.80  120.86  59.35  116.65  7.61  194.20  183.48  125.90  79.85
8445  03-Dec-2018 09:01  184.15  48.50  50.07  112.80  147.75  112.94  49.93  190.40  211.36  76.85  46.51  94.73  130.48  285.50  123.50  144.10  59.83  99.00  80.65  113.59  361.0  141.80  120.86  59.35  116.50  7.61  194.20  183.48  125.90  79.85
5580  03-Dec-2018 09:02  184.01  48.45  50.06  112.98  147.75  112.94  49.93  190.40  211.36  76.85  46.51  94.73  130.48  285.50  123.50  144.10  59.84  99.00  80.65  113.59  360.8  1

In [17]:
!head prices.csv
!tail prices.csv

Date,AAPL,CSCO,INTC,MSFT,JNJ,JPM,KO,MCD,MMM,NKE,PFE,PG,TRV,UNH,UTX,V,VZ,WMT,XOM,AXP,BA,CAT,CVX,DWDP,DIS,GE,GS,HD,IBM,MRK
03-Dec-2018 09:00,183.97,48.45,50.07,112.8,147.75,112.94,49.93,190.4,211.36,76.85,46.51,94.73,130.48,285.5,123.5,144.1,59.83,99.0,80.65,113.59,361.0,141.8,120.86,59.35,116.65,7.61,194.2,183.48,125.9,79.85
03-Dec-2018 09:01,184.15,48.5,50.07,112.8,147.75,112.94,49.93,190.4,211.36,76.85,46.51,94.73,130.48,285.5,123.5,144.1,59.83,99.0,80.65,113.59,361.0,141.8,120.86,59.35,116.5,7.61,194.2,183.48,125.9,79.85
03-Dec-2018 09:02,184.01,48.45,50.06,112.98,147.75,112.94,49.93,190.4,211.36,76.85,46.51,94.73,130.48,285.5,123.5,144.1,59.84,99.0,80.65,113.59,360.8,141.8,120.86,59.35,116.5,7.61,194.2,183.48,125.88,79.85
03-Dec-2018 09:03,184.0,48.45,50.08,112.96,147.75,112.94,49.93,190.4,211.36,76.85,46.51,94.73,130.48,285.5,123.5,144.1,59.84,98.79,80.65,113.59,360.5,141.9,120.86,59.35,116.51,7.62,194.2,183.48,125.88,79.85
03-Dec-2018 09:04,183.99,48.55,50.1,112.91,146.9,112.9,49.